In [1]:
import polars as pl 
import numpy as np 
from  matplotlib import pyplot as plt  
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold



In [53]:
(model_data := pl.read_csv("water_quality_and_parcel_summaries_2004_to_2015 (1).csv"))

,Monit_MAP_CODE1,Year,LAKE_NAME,Mean_Secchi_Depth_Result,Mean_Phosporus_Result,Mean_EMV_Total,STD_EMV_Total,Mean_Sale_Value,STD_Sale_Value,Mean_Total_Tax,STD_Total_Tax,Mean_Garage_Size,STD_Garage_Size,Mean_Fin_SQ_FT,STD_Fin_SQ_FT,Percentage_Yes_Basement,Percentage_Yes_Garage,Percentage_Yes_Tax_Exempt,Percentage_Air_Cooling,Percentage_AC_Cooling,Percentage_Central_Cooling,Percentage_Other_Cooling,Percentage_No_Cooling,Percentage_Air_Heating,Percentage_Space_Heating,Percentage_Water_Heating,Percentage_Electric_Heating,Percentage_Other_Heating,Percentage_No_Heating
i64,str,i64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0,"""02000500-01""",2014,"""George Watch Lake""",0.716667,0.108778,215637.592745,459023.922281,122485.615829,183414.899771,3313.685903,17837.814516,null,null,1849.473207,8220.487298,70.486397,0.0,7.914262,0.0,0.0,0.0,2.308326,97.691674,0.0,0.0,0.0,0.0,85.325639,14.674361
1,"""02000500-01""",2013,"""George Watch Lake""",0.365,0.3105,196764.633141,454192.377508,120601.723001,190129.461399,3319.081616,18199.952917,null,null,1841.112119,8221.467234,69.991756,0.0,7.914262,0.824402,0.0,0.0,1.154163,0.0,79.060181,0.824402,1.154163,2.802968,1.154163,0.0
2,"""02000500-01""",2012,"""George Watch Lake""",0.359,0.2649,200414.333057,480355.968668,118315.128418,190091.029827,3460.064623,19346.923479,null,null,1811.576636,8223.512958,70.173985,0.0,8.119304,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"""02000500-01""",2011,"""George Watch Lake""",0.973333,0.119417,216297.932175,536941.029067,111218.812242,190312.451914,3459.313482,19320.28833,null,null,1780.282051,8215.092878,76.840364,0.0,8.271299,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"""02000500-01""",2010,"""George Watch Lake""",0.493333,0.173,222789.430223,559656.083477,110765.119736,190448.272924,3384.645747,18646.626107,null,null,1760.876135,8210.519477,75.887696,0.0,8.092486,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
523,"""82036800-01""",2008,"""Klawitter Pond""",0.596923,0.092462,420461.350844,802637.556411,128391.380863,182645.51138,412444.840525,798052.142235,null,null,0.0,0.0,0.0,0.0,1.876173,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
524,"""82036800-01""",2007,"""Klawitter Pond""",0.491667,0.098083,396113.320826,487241.389735,131310.281426,178831.780564,0.0,0.0,null,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
525,"""82036800-01""",2006,"""Klawitter Pond""",0.6,0.1035,371583.114447,261100.184471,120477.452158,168187.904304,2743.87242,1971.437642,null,null,1634.529081,1160.042283,0.0,0.0,1.125704,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [56]:
(lakes := model_data.select("Monit_MAP_CODE1")
                    .unique()
                    .to_series()
                    .to_list()
                    )


['19034800-01',
 '19002400-01',
 '82011301-01',
 '10001900-01',
 '82033400-01',
 '19002500-01',
 '19044600-01',
 '10001100-01',
 '27071100-01',
 '82008900-01',
 '82010400-01',
 '82036800-01',
 '82003400-01',
 '02000500-01',
 '82009700-01',
 '10000200-01',
 '82010300-01',
 '82012300-01',
 '82007700-01',
 '82010100-01',
 '10009500-01',
 '27004201-01',
 '27062700-01',
 '82011602-01',
 '19002700-01',
 '19002601-01',
 '19002900-01',
 '82015900-01',
 '19003300-01',
 '27007000-01',
 '82009200-01',
 '19002300-01',
 '82013700-01',
 '70002600-01',
 '19003100-01',
 '82015300-01',
 '27005300-01',
 '82012200-01',
 '82005400-01',
 '82008700-01',
 '82009400-01',
 '10005200-01',
 '19002200-01',
 '19002100-01',
 '10012100-01',
 '82002000-01',
 '82009002-01',
 '27003501-01']

In [4]:
np.random.seed(123)

validation_ids = np.random.choice(
    lakes,
    size=int(0.30 * len(lakes)),
    replace=False
)

In [5]:
remaining = np.setdiff1d(lakes, validation_ids)

train_ids = np.random.choice(
    remaining,
    size=int(0.70 * len(remaining)),
    replace=False
)

test_ids = np.setdiff1d(remaining, train_ids)


In [6]:
(model_data_2 := model_data.with_columns(
    pl.when(pl.col("Monit_MAP_CODE1").is_in(validation_ids))
      .then(pl.lit("Validation"))
      .when(pl.col("Monit_MAP_CODE1").is_in(train_ids))
      .then(pl.lit("Training"))
      .otherwise(pl.lit("Test"))
      .alias("TrainingValidation")
)
)

,Monit_MAP_CODE1,Year,LAKE_NAME,Mean_Secchi_Depth_Result,Mean_Phosporus_Result,Mean_EMV_Total,STD_EMV_Total,Mean_Sale_Value,STD_Sale_Value,Mean_Total_Tax,STD_Total_Tax,Mean_Garage_Size,STD_Garage_Size,Mean_Fin_SQ_FT,STD_Fin_SQ_FT,Percentage_Yes_Basement,Percentage_Yes_Garage,Percentage_Yes_Tax_Exempt,Percentage_Air_Cooling,Percentage_AC_Cooling,Percentage_Central_Cooling,Percentage_Other_Cooling,Percentage_No_Cooling,Percentage_Air_Heating,Percentage_Space_Heating,Percentage_Water_Heating,Percentage_Electric_Heating,Percentage_Other_Heating,Percentage_No_Heating,TrainingValidation
i64,str,i64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str
0,"""02000500-01""",2014,"""George Watch Lake""",0.716667,0.108778,215637.592745,459023.922281,122485.615829,183414.899771,3313.685903,17837.814516,null,null,1849.473207,8220.487298,70.486397,0.0,7.914262,0.0,0.0,0.0,2.308326,97.691674,0.0,0.0,0.0,0.0,85.325639,14.674361,"""Test"""
1,"""02000500-01""",2013,"""George Watch Lake""",0.365,0.3105,196764.633141,454192.377508,120601.723001,190129.461399,3319.081616,18199.952917,null,null,1841.112119,8221.467234,69.991756,0.0,7.914262,0.824402,0.0,0.0,1.154163,0.0,79.060181,0.824402,1.154163,2.802968,1.154163,0.0,"""Test"""
2,"""02000500-01""",2012,"""George Watch Lake""",0.359,0.2649,200414.333057,480355.968668,118315.128418,190091.029827,3460.064623,19346.923479,null,null,1811.576636,8223.512958,70.173985,0.0,8.119304,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""Test"""
3,"""02000500-01""",2011,"""George Watch Lake""",0.973333,0.119417,216297.932175,536941.029067,111218.812242,190312.451914,3459.313482,19320.28833,null,null,1780.282051,8215.092878,76.840364,0.0,8.271299,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""Test"""
4,"""02000500-01""",2010,"""George Watch Lake""",0.493333,0.173,222789.430223,559656.083477,110765.119736,190448.272924,3384.645747,18646.626107,null,null,1760.876135,8210.519477,75.887696,0.0,8.092486,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""Test"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
523,"""82036800-01""",2008,"""Klawitter Pond""",0.596923,0.092462,420461.350844,802637.556411,128391.380863,182645.51138,412444.840525,798052.142235,null,null,0.0,0.0,0.0,0.0,1.876173,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""Validation"""
524,"""82036800-01""",2007,"""Klawitter Pond""",0.491667,0.098083,396113.320826,487241.389735,131310.281426,178831.780564,0.0,0.0,null,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""Validation"""
525,"""82036800-01""",2006,"""Klawitter Pond""",0.6,0.1035,371583.114447,261100.184471,120477.452158,168187.904304,2743.87242,1971.437642,null,null,1634.529081,1160.042283,0.0,0.0,1.125704,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""Validation"""


In [7]:
training_data = model_data_2.filter(pl.col("TrainingValidation") == "Training")
validation_data = model_data_2.filter(pl.col("TrainingValidation") == "Validation")
test_data = model_data_2.filter(pl.col("TrainingValidation") == "Test")

In [8]:
model_data_2.select(pl.col("TrainingValidation").value_counts())

TrainingValidation
struct[2]
"{""Training"",253}"
"{""Validation"",154}"
"{""Test"",121}"


In [9]:
drop_cols = {
    "Mean_Secchi_Depth_Result",          
    "Mean_Phosporus_Result",      
    "TrainingValidation",        
    "Monit_MAP_CODE1",                 
    "LAKE_NAME",
    "Year"
}


for avg_secchi_depth

In [10]:

X_train = training_data.drop(drop_cols).to_pandas()
y_train = training_data.select("Mean_Secchi_Depth_Result").to_pandas()

X_val = validation_data.drop(drop_cols).to_pandas()
y_val = validation_data["Mean_Secchi_Depth_Result"].to_pandas()

X_test = test_data.drop(drop_cols).to_pandas()
y_test = test_data.select("Mean_Secchi_Depth_Result").to_pandas()

for avg_total_phosphorus

In [11]:
X_train_tp = training_data.drop(drop_cols).to_pandas()
y_train_tp = training_data["Mean_Phosporus_Result"].to_pandas()

X_val_tp = validation_data.drop(drop_cols).to_pandas()
y_val_tp = validation_data["Mean_Phosporus_Result"].to_pandas()

X_test_tp = test_data.drop(drop_cols).to_pandas()
y_test_tp = test_data.select("Mean_Phosporus_Result").to_pandas()

## grid search 

In [12]:
import numpy as np

param_grid = {
    'max_depth': [None, 1, 2, 3, 4, 5],
    'min_samples_split': [2, 3, 5, 10],
    'min_samples_leaf': [1, 5, 10]
}

rf_params = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 10, None],
    'min_samples_split': [2, 3, 5, 10],
    'min_samples_leaf': [1, 5, 10],
    'max_features': ['sqrt', 'log2']
}

In [13]:
from sklearn.model_selection import StratifiedKFold
state = 45854

cv_obj = KFold(n_splits=10, shuffle=True, random_state=state)
cv_obj

KFold(n_splits=10, random_state=45854, shuffle=True)

In [14]:
grid_search_1 = GridSearchCV(DecisionTreeRegressor(), param_grid, verbose=3, cv=cv_obj)

In [15]:
grid_search_rf = GridSearchCV(RandomForestRegressor(), rf_params, verbose=3, cv=cv_obj, n_jobs=-1)

## training for avg_secchi_depth

#### DecisionTreeRegressor

In [16]:
grid_search_1.fit(X_train, y_train)

Fitting 10 folds for each of 72 candidates, totalling 720 fits
[CV 1/10] END max_depth=None, min_samples_leaf=1, min_samples_split=2;, score=0.731 total time=   0.0s
[CV 2/10] END max_depth=None, min_samples_leaf=1, min_samples_split=2;, score=-0.008 total time=   0.0s
[CV 3/10] END max_depth=None, min_samples_leaf=1, min_samples_split=2;, score=0.537 total time=   0.0s
[CV 4/10] END max_depth=None, min_samples_leaf=1, min_samples_split=2;, score=0.826 total time=   0.0s
[CV 5/10] END max_depth=None, min_samples_leaf=1, min_samples_split=2;, score=0.804 total time=   0.0s
[CV 6/10] END max_depth=None, min_samples_leaf=1, min_samples_split=2;, score=0.746 total time=   0.0s
[CV 7/10] END max_depth=None, min_samples_leaf=1, min_samples_split=2;, score=-0.073 total time=   0.0s
[CV 8/10] END max_depth=None, min_samples_leaf=1, min_samples_split=2;, score=0.833 total time=   0.0s
[CV 9/10] END max_depth=None, min_samples_leaf=1, min_samples_split=2;, score=0.714 total time=   0.0s
[CV 10/1

,estimator,DecisionTreeRegressor()
,param_grid,"{'max_depth': [None, 1, ...], 'min_samples_leaf': [1, 5, ...], 'min_samples_split': [2, 3, ...]}"
,scoring,None
,n_jobs,None
,refit,True
,cv,KFold(n_split... shuffle=True)
,verbose,3
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,criterion,'squared_error'


In [17]:
grid_search_1.best_score_
grid_search_1.best_params_

{'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5}

In [18]:
(best_model_1 := grid_search_1.best_estimator_)

,criterion,'squared_error'
,splitter,'best'
,max_depth,None
,min_samples_split,5
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,None
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,ccp_alpha,0.0


In [19]:
cart_test_pred = best_model_1.predict(X_test)

## Testing DecisionTreeRegressor - avg_secchi_depth

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(y_test, cart_test_pred)
rmse = np.sqrt(mse)
(r2 := r2_score(y_test, cart_test_pred))

-0.8674787380597262

#### RandomForestRegressor

In [21]:
grid_search_rf.fit(X_train, y_train)

Fitting 10 folds for each of 216 candidates, totalling 2160 fits


c:\Users\ym5765bo\AppData\Local\anaconda3\envs\polars\Lib\site-packages\sklearn\base.py:1365: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


,estimator,RandomForestRegressor()
,param_grid,"{'max_depth': [5, 10, ...], 'max_features': ['sqrt', 'log2'], 'min_samples_leaf': [1, 5, ...], 'min_samples_split': [2, 3, ...], ...}"
,scoring,None
,n_jobs,-1
,refit,True
,cv,KFold(n_split... shuffle=True)
,verbose,3
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,n_estimators,100


In [22]:
grid_search_rf.best_score_

np.float64(0.6894344824564519)

In [23]:
grid_search_rf.best_params_

{'max_depth': None,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 100}

In [24]:
(best_model_rf := grid_search_rf.best_estimator_)

,n_estimators,100
,criterion,'squared_error'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


## Testing RandomForestRegressor  - avg-secchi_depth

In [25]:
y_test_pred_2 = best_model_rf.predict(X_test)

In [26]:

val_r2 = r2_score(y_test, y_test_pred_2)
val_r2

-0.3624176544849296

## training for avg_total_phosphorus      


#### DecisionTreeRegressor

In [27]:
grid_search_1_tp=grid_search_1.fit(X_train_tp, y_train_tp)

Fitting 10 folds for each of 72 candidates, totalling 720 fits
[CV 1/10] END max_depth=None, min_samples_leaf=1, min_samples_split=2;, score=0.447 total time=   0.0s
[CV 2/10] END max_depth=None, min_samples_leaf=1, min_samples_split=2;, score=0.391 total time=   0.0s
[CV 3/10] END max_depth=None, min_samples_leaf=1, min_samples_split=2;, score=0.637 total time=   0.0s
[CV 4/10] END max_depth=None, min_samples_leaf=1, min_samples_split=2;, score=0.731 total time=   0.0s
[CV 5/10] END max_depth=None, min_samples_leaf=1, min_samples_split=2;, score=0.675 total time=   0.0s
[CV 6/10] END max_depth=None, min_samples_leaf=1, min_samples_split=2;, score=0.284 total time=   0.0s
[CV 7/10] END max_depth=None, min_samples_leaf=1, min_samples_split=2;, score=0.604 total time=   0.0s
[CV 8/10] END max_depth=None, min_samples_leaf=1, min_samples_split=2;, score=0.333 total time=   0.0s
[CV 9/10] END max_depth=None, min_samples_leaf=1, min_samples_split=2;, score=0.683 total time=   0.0s
[CV 10/10]

In [28]:
grid_search_1_tp.best_score_
grid_search_1_tp.best_params_

{'max_depth': None, 'min_samples_leaf': 5, 'min_samples_split': 2}

In [29]:
(best_model_1_tp := grid_search_1_tp.best_estimator_)

,criterion,'squared_error'
,splitter,'best'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,5
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,None
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,ccp_alpha,0.0


In [30]:
y_test_pred_1_tp = best_model_1_tp.predict(X_test_tp)

## Testing DecisionTreeRegressor - avg_total_phosphorus

In [50]:
mse = mean_squared_error(y_test, y_test_pred_1_tp)
rmse = np.sqrt(mse)
(r2:= r2_score(y_test_tp, y_test_pred_1_tp))

-0.2872461865320002

#### RandomForestRegressor

In [32]:
grid_search_rf_tp= grid_search_rf.fit(X_train_tp, y_train_tp)

Fitting 10 folds for each of 216 candidates, totalling 2160 fits


In [33]:
grid_search_rf_tp.best_score_

np.float64(0.6743276090593142)

In [34]:
grid_search_rf_tp.best_params_

{'max_depth': None,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 3,
 'n_estimators': 100}

In [35]:
(best_model_rf_tp := grid_search_rf_tp.best_estimator_)

,n_estimators,100
,criterion,'squared_error'
,max_depth,None
,min_samples_split,3
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


## Testing RandomforestRegressor - avg_total_phosphorus

In [36]:
y_test_pred_2_tp = best_model_rf_tp.predict(X_test_tp)

In [37]:
(r2_rf := r2_score(y_test_tp, y_test_pred_2_tp))

-0.11814519791095202

## Best models 

For both targets, the Random Forest models outperformed the CART models. The best performing models were best_model_rf and grid_search_rf_tp

combine train and test to refit the best model

In [39]:
final_train = training_data.vstack(test_data)
final_X= final_train.drop(drop_cols).to_pandas()
final_y = final_train.select("Mean_Secchi_Depth_Result").to_pandas()

In [40]:
best_model_rf_final=best_model_rf.fit(final_X, final_y)

c:\Users\ym5765bo\AppData\Local\anaconda3\envs\polars\Lib\site-packages\sklearn\base.py:1365: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [41]:
y_val_pred = best_model_rf_final.predict(X_val)

In [51]:
(val_r2 := r2_score(y_val, y_val_pred))


0.16541934817230386

In [44]:

final_X_tp= final_train.drop(drop_cols).to_pandas()
final_y_tp = final_train.select("Mean_Phosporus_Result").to_pandas()

In [45]:
best_model_rf_tp_final=best_model_rf_tp.fit(final_X_tp, final_y_tp)

c:\Users\ym5765bo\AppData\Local\anaconda3\envs\polars\Lib\site-packages\sklearn\base.py:1365: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [46]:
y_val_pred_tp = best_model_rf_tp_final.predict(X_val)

In [ ]:
(val_r2_tp := r2_score(y_val, y_val_pred_tp))

-2.088544637904972

I compared CART and Random Forest for each target, and Random Forest performed better both times. After picking it as the best model, I retrained it using the combined training and test lakes, then evaluated it on the validation lakes. The final results weren’t great Secchi depth had an R² of about 0.17, and phosphorus was negative (around –2.09), meaning the model didn’t generalize well. But the workflow itself was done correctly: choose the best model, retrain it, and test it on completely unseen lakes.